<a href="https://colab.research.google.com/github/apurwanto04/AJ_GIS/blob/master/surface_water_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Surface water mapping

## Setup

In [ ]:
from google.colab import drive
import ee
import geemap
import geopandas as gpd
import glob
import datetime
import pandas as pd

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/surface_water_mapping

/content/drive/MyDrive/surface_water_mapping


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
user_id = 'ee-ajuligee-coleb'
ee.Authenticate()
ee.Initialize(project=user_id)

In [8]:
folder_name = 'surface_water_mapping'

In [9]:
# decide whether to export vectors to Drive
# True: if you haven't exported them yet
# False: if you only want to create the final shapefile
export = False

## Define date range of interest and region of interest

In [10]:
start_date = '2016-11-01'
end_date = '2017-04-30'

In [11]:
# Makgadikgadi Pan
roi_name = 'makgadikgadi_pan'

long_min = 24.51122
long_max = 25.41764
lat_min = -20.7972
lat_max = -20.40985

roi = ee.Geometry.Polygon([[
    [long_min, lat_max],
    [long_min, lat_min],
    [long_max, lat_min],
    [long_max, lat_max]
]])

## Load Sentinel-1 collection

In [12]:
col_filter = ee.Filter.And(
    ee.Filter.date(ee.Date(start_date), ee.Date(end_date)),
    ee.Filter.bounds(roi),
    ee.Filter.eq('instrumentMode', 'IW'),
    ee.Filter.eq('orbitProperties_pass', 'ASCENDING'),
    ee.Filter.eq('resolution_meters', 10),
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')
)

s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filter(col_filter)

In [13]:
s1

In [14]:
m = geemap.Map()
m.center_object(roi, 10)
m.add_layer(
    s1.first().clip(roi),
    {'bands': 'VV', 'min': -20, 'max': 0},
    'S1 image',
)
m

Map(center=[-20.60403337455113, 24.96442999999993], controls=(WidgetControl(options=['position', 'transparent_…

## Filter speckle noise

In [16]:
def filter_speckles(image):
  vv = image.select('VV')
  vv_smoothed = vv.focal_median(50, 'circle', 'meters').rename('VV_smoothed')
  return image.addBands(vv_smoothed)

In [17]:
s1_filtered = s1.map(filter_speckles)

In [18]:
s1_filtered

## Classify water

In [19]:
def classify_water(image):
  vv_smoothed = image.select('VV_smoothed')
  water = vv_smoothed.lt(-16.5616).rename('water')
  water = water.updateMask(water)
  return image.addBands(water)

In [20]:
s1_classified = s1_filtered.map(classify_water)

In [21]:
s1_classified

In [22]:
m = geemap.Map()
m.center_object(roi, 10)
m.add_layer(
    s1_filtered.first().clip(roi),
    {'bands': 'VV', 'min': -20, 'max': 0},
    'Filtered S1 image',
)
m.add_layer(
    s1_classified.first().clip(roi),
    {'bands': 'water', 'min': 0, 'max': 1, 'palette': ['#FFFFFF', '#0000FF']},
    'Water',
)
m

Map(center=[-20.60403337455113, 24.96442999999993], controls=(WidgetControl(options=['position', 'transparent_…

## Reduce classified images to vectors and start export

In [23]:
s1_classified_list = s1_classified.toList(s1_classified.size())
s1_classified_list_size = s1_classified_list.size().getInfo()

In [24]:
if export:
  for i in range(s1_classified_list_size):

    print('Processing image {}/{}...'.format(i+1, s1_classified_list_size))

    # get ith image from collection
    image = ee.Image(s1_classified_list.get(i))

    # generate file name
    date = image.date().format("YYYY-MM-dd").getInfo().replace('-', '')
    file_name = roi_name + '_' + date

    # reduce image to vectors
    vectors = image.select('water').reduceToVectors(
      reducer=ee.Reducer.countEvery(),
      geometry=roi,
      scale=10,
      geometryType='polygon',
      labelProperty='water',
      crs='EPSG:4326',
      maxPixels=1e13
    )

    # export vectors to shapefile
    task = ee.batch.Export.table.toDrive(
      collection=vectors,
      description=file_name,
      folder=folder_name,
      fileFormat='SHP'
    )
    task.start()

You can check the progress of your export here: https://code.earthengine.google.com/tasks

## Load and combine exported shape files
Make sure all shape files have been successfully exported before trying to combine them into one single shape file.

In [25]:
shape_files = glob.glob('*.shp')

In [26]:
# make sure final shapefile is skipped here if it already exits
length_file_name_final = len(roi_name + '_' + start_date.replace('-', '')) + 9 + 4 # add end date and file extension to file name length
shape_files = [i for i in shape_files if len(i) < length_file_name_final]

In [27]:
len(shape_files)

0

In [28]:
shape_files_gdf = gpd.GeoDataFrame(columns=['geometry', 'date'], geometry='geometry')

for shape_file in shape_files:
  shape_file_gdf_temp = gpd.read_file(shape_file)
  shape_file_gdf_temp['date'] = str(datetime.datetime.strptime(shape_file[17:25], "%Y%m%d").date())
  shape_file_gdf_temp = shape_file_gdf_temp.drop(['count', 'water'], axis=1)
  shape_files_gdf = pd.concat([shape_files_gdf, shape_file_gdf_temp])

In [29]:
shape_files_gdf = shape_files_gdf.reset_index(drop=True)

In [30]:
shape_files_gdf

,geometry,date


In [31]:
shape_files_gdf.groupby('date').size()

,0
date,


In [32]:
# identify invalid geometries
shape_files_gdf['is_valid'] = shape_files_gdf['geometry'].is_valid
print("Number of invalid geometries:", shape_files_gdf[~shape_files_gdf['is_valid']].shape[0])

Number of invalid geometries: 0


In [33]:
# repair invalid geometries
invalid_mask = ~shape_files_gdf['geometry'].is_valid
shape_files_gdf.loc[invalid_mask, 'geometry'] = shape_files_gdf.loc[invalid_mask, 'geometry'].buffer(0)

In [34]:
# recheck for validity
assert shape_files_gdf['geometry'].is_valid.all(), "Some geometries are still invalid!"

In [35]:
file_name_final = roi_name + '_' + str(min(shape_files_gdf['date'])).replace('-', '') + '_' + str(max(shape_files_gdf['date'])).replace('-', '') + '.shp'
shape_files_gdf.to_file(file_name_final)

ValueError: min() arg is an empty sequence